## LeNet-5 实现<br>
由卷积编码器和全连接层密集快组成

In [26]:
import torch
from torch import nn
from d2l import torch as d2l

In [27]:
class Reshape(nn.Module):
    def forward(self, x):
        return x.reshape(-1, 1, 28, 28) # 本层就是将输入数据reshape成（批量大小，通道数，宽，高） 应该是这样

net = nn.Sequential(
    Reshape(), nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2), nn.Sigmoid(), # padding的原因是此处原图为28（不同于原始LeNet的32）
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(), # 后续的全连接层的输入需要是一维的向量，故将4D->2D(保留了批的那一维)
    nn.Linear(16 * 5 * 5, 120),  nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [28]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t\t', X.shape)

""" >>> 第一个维度都是批量大小
第一层将通道数增加到6
第二层将28池化为14
第三层将通道数增加到16
第四层将10池化到5
第五层全连接层（不同于LeNet-5中的卷积层？）

"""

Reshape output shape:		 torch.Size([1, 1, 28, 28])
Conv2d output shape:		 torch.Size([1, 6, 28, 28])
Sigmoid output shape:		 torch.Size([1, 6, 28, 28])
AvgPool2d output shape:		 torch.Size([1, 6, 14, 14])
Conv2d output shape:		 torch.Size([1, 16, 10, 10])
Sigmoid output shape:		 torch.Size([1, 16, 10, 10])
AvgPool2d output shape:		 torch.Size([1, 16, 5, 5])
Flatten output shape:		 torch.Size([1, 400])
Linear output shape:		 torch.Size([1, 120])
Sigmoid output shape:		 torch.Size([1, 120])
Linear output shape:		 torch.Size([1, 84])
Sigmoid output shape:		 torch.Size([1, 84])
Linear output shape:		 torch.Size([1, 10])


' >>> 第一个维度都是批量大小\n第一层将通道数增加到6\n'

## LeNet在Fashion-MNIST上的表现

In [35]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [ ]:
def evalute_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device
    
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            